In [16]:
'''
实验目的：实现基于CNN、RNN的文本分类

实验内容：
1）词嵌入初始化方式：随机embedding、word2vec、word2vec_fun-tured、加载glove
2）CNN/RNN的特征抽取
3）Dropout


补充：
1）
2）

参考：
https://arxiv.org/abs/1408.5882



'''

'\n实验目的：实现基于CNN、RNN的文本分类\n\n实验内容：\n1）词嵌入初始化方式：随机embedding、word2vec、word2vec_fun-tured、加载glove\n2）CNN/RNN的特征抽取\n3）Dropout\n\n\n补充：\n1）\n2）\n\n参考：\nhttps://arxiv.org/abs/1408.5882\n\n\n\n'

In [17]:
import os
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import sklearn

dir_all_data='D:\\workspace\\nlp_beginer_solution\\data\\task2_all_data.tsv'

In [18]:
print(os.getcwd())
data_all=pd.read_csv(dir_all_data,sep='\t')
#print(all_data.shape)    #(156060, 4)
#print(all_data.keys())   #['PhraseId', 'SentenceId', 'Phrase', 'Sentiment']
x_all=data_all['Phrase']
y_all=data_all['Sentiment']
print(x_all.shape)

D:\workspace\nlp_beginer_solution
(156060,)


In [19]:
#划分验证集、测试集
from sklearn.model_selection import train_test_split
x_train, x_test,y_train,  y_test = train_test_split(x_all,y_all, test_size=0.2)
x_train, x_val,y_train,y_val = train_test_split(x_train,y_train, test_size=0.25)
print(x_train.shape, x_val.shape, x_test.shape)

(93636,) (31212,) (31212,)


In [20]:
#将Series类型的数据转换为List
x_train=[sentence.split() for sentence in x_train]
x_test=[sentence.split() for sentence in x_test]
x_val=[sentence.split() for sentence in x_val]


In [21]:
#将pandas的句子级别的Series转化成词级别的List
import itertools
words_list = list(itertools.chain.from_iterable(x_train))
print(len(words_list))
words_list=list(set(words_list))
print(len(words_list))

674680
18083


In [22]:
#构建word 与 id的转换
word2id={word:index for index,word in enumerate(words_list)}
id2word={index:word for index,word in enumerate(words_list)}

In [16]:
embedding_choice='rand'
#embedding_choice='static'
#embedding_choice='non-static'
num_embeddings =10000
embedding_dim =100
dropout_p=0.5
filters_num=100
label_num=5

class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.embedding_choice=embedding_choice
        self.vocab_size=vocab_size
        
        if self.embedding_choice==  'rand':
            self.embedding=nn.Embedding(num_embeddings,embedding_dim)
            
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=filters_num ,  #卷积产生的通道
                               kernel_size=(3, embedding_dim), padding=(2,0))
        
        self.conv2 = nn.Conv2d(in_channels=1,out_channels=filters_num ,  #卷积产生的通道
                               kernel_size=(4, embedding_dim), padding=(3,0))
        
        self.conv3 = nn.Conv2d(in_channels=1,out_channels=filters_num ,  #卷积产生的通道
                               kernel_size=(5, embedding_dim), padding=(4,0))
        
        self.dropout = nn.Dropout(dropout_p)
        
        self.fc = nn.Linear(filters_num * 3, label_num)
        
    def forward(self,x):
        x=self.embedding(x)
        
        
        return x
        

In [13]:
num_embeddings =10000
embedding_dim =100
dropout_p=0.5
filters_num=100
label_num=5



embedding=nn.Embedding(num_embeddings,embedding_dim)

conv1 = nn.Conv2d(in_channels=1,out_channels=filters_num ,  #卷积产生的通道
                       kernel_size=(3, embedding_dim), padding=(2,0))

conv2 = nn.Conv2d(in_channels=1,out_channels=filters_num ,  #卷积产生的通道
                       kernel_size=(4, embedding_dim), padding=(3,0))

conv3 = nn.Conv2d(in_channels=1,out_channels=filters_num ,  #卷积产生的通道
                       kernel_size=(5, embedding_dim), padding=(4,0))

dropout = nn.Dropout(dropout_p)

fc = nn.Linear(filters_num * 3, label_num)





In [ ]:
x_train_id = torch.from_numpy(a)



In [25]:
print(x_train[0])
x=embedding(torch.LongTensor(x_train))
print(x)

['a', 'reluctant', ',', 'irresponsible', 'man']


ValueError: too many dimensions 'str'